## Lab 03: Project Gutenberg Books (2/2)

In this lab we will continue working with books from the [Project Gutenberg](https://www.gutenberg.org/) collection.  We'll begin by discussing regular expressions, and then use these beautiful little gadgets to try to automatically find the main characters in a book. We'll apply this to the books from last week, and then try it on some new books.

<br>



In [3]:
from datascience import *
import numpy as np
import re

# direct plots to appear within the cell, and set their style
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')


Regular expressions are a mainstay of working with text. They can be used to *quickly* find matches of expressions in strings, but with `wildcard' characters so that we don't need to know exactly what we're searching for. In a technical sense, a regular expression takes a pattern and automatically builds a special-purpose little program that is used to search text to match that pattern. We won't discuss the details of how these work&mdash;this is done in most beginning theoretical computer science courses&mdash;but we'll get some practice using them in Python.


Here is a table of some of the primary syntax used for regular expressions in Python. The syntax is similar, but usually a little bit different, across different programming languages.

| Expression| Description |
| --- | --- | 
| `^` | Matches the expression to its right at the start of a string |
| `$` | Matches the expression to its left at the end of a string |
| `.` | Matches any character except line terminators like `\n` |
| `+` | Greedily matches the expression to its left 1 or more times |
| `*` | Greedily matches the expression to its left 0 or more times |
| `?` | Greedily matches the expression to its left 0 or 1 times |
| `\w` | Matches alphanumeric characters |
| `\d` | Matches digits 0-9 |
| `\D` | Matches any non-digits |
| `\s` | Matches whitespace characters |
| `\S` | Matches non-whitespace characters |
| `[ ]` | Contains a set of characters to match 
| `[amk]` | Matches either a, m, or k. It does not match amk|
| `[a-z]` | Matches any letter from a to z|
| `[a-z0-9]` | Matches characters from a to z and also from 0 to 9 |
| `[^ab5]` | Matches characters that are not a, b, or 5|


So, the expression `\w+` matches any alphanumeric string of length at least one character, excluding whitespace. Here is an example of a regular expression that tokenizes a string into words.

In [4]:
result = re.findall('\w+', 'this is a test')
print(result)

# you can also `compile' the expression into a piece of code

pattern = re.compile('\w+')
result = pattern.findall('this is a test')
print(result)

['this', 'is', 'a', 'test']
['this', 'is', 'a', 'test']


If we want to match only the first word of the string, we prefix by the `^` character, which means beginning-of-string.

In [5]:
result = re.findall('^\w+', 'this is a test')
print(result)


['this']


We can also use the `re.split` function to divide up a word by anything matching a regular expression.


In [6]:
re.split('\s+', 'this is      a test')

['this', 'is', 'a', 'test']

Note the difference if we only match a single space, changing the expression to `\s`


In [7]:
re.split('\s', 'this is      a test')

['this', 'is', '', '', '', '', '', 'a', 'test']

You can test regular expressions using the web site [regex101.com](http://regex101.com). You may also find this [regex cheat sheet](https://www.dataquest.io/blog/regex-cheatsheet/) handy.

The three primary methods in `re` are `re.match`, `re.search`, and `re.findall`. The difference between them is that 
`re.match` searches the pattern from the start of the string, `re.search` finds the *first* occurrence, while `re.findall` looks for occurrences of the pattern *anywhere* in the string.  Let's explore these a bit.


In [8]:
print("re.findall: %s" % re.findall('[A-Z]\w+', '"This is a test," Mr. Smith said to Jones'))
print("re.search: %s" % re.search('[A-Z]\w+', '"This is a test," Mr. Smith said to Jones'))
print("re.match: %s" % re.match('[A-Z]\w+', '"This is a test," Mr. Smith said to Jones'))

re.findall: ['This', 'Mr', 'Smith', 'Jones']
re.search: <_sre.SRE_Match object; span=(1, 5), match='This'>
re.match: None


Since we will be focusing on finding all of the characters in a book, we'll focus on the use of `re.findall`, but the regular expressions that can be used in all of these are the same. Ok, so now let's try out one of these gadgets on Huckleberry Finn. Let's first load the book into a big string.



In [9]:
from urllib.request import urlopen 
import re
def read_url(url): 
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

Recall that the online book for "Adventures of Huckleberry Finn," by Mark Twain, is [here](https://www.gutenberg.org/ebooks/76).
From this web site you can see various metadata for the book as well as the [link the text itself](https://www.gutenberg.org/files/76/76-0.txt), which is [https://www.gutenberg.org/files/76/76-0.txt](https://www.gutenberg.org/files/76/76-0.txt)


In [10]:
huck_finn_url = 'https://www.gutenberg.org/files/76/76-0.txt'
huck_finn_text = read_url(huck_finn_url)


The following function takes a book url, opens it to get a text string, and then finds all occurrences of "said NAME" using regular expressions. For each name that it finds, it re-searches the string to count all occurrences of that name. Finally a `datascience.Table` is constructed with those data, which is then sorted in decreasing order

In [18]:

def find_characters(book_url):
    book_text = read_url(book_url)
    names = {name for name in re.findall('said ([A-Z]\w+)', book_text)}
    counts = [len(re.findall("(%s)\s" % name, book_text)) for name in names]
    characters = Table().with_columns('Name', names, 'Count', counts)
    return characters.sort('Count', descending=True)

url = "https://www.gutenberg.org/files/76/76-0.txt"
characters = find_characters(url)
characters

Name,Count
Jim,255
Tom,175
Aunt,56
Sherburn,12


Can we do better? Of course we can! In the following, we will explore regular expressions that use different types of phrases like "said NAME", include "NAME said", and also look for `honorific' expressions like Dr. Doolittle.


In [12]:
dialogue = re.findall('(said|told) ([A-Z]\w+)', '"hi" said John, "yo!" told Joan! said Steve')
names1 = {name[1] for name in dialogue}

dialogue = re.findall('([A-Z]\w+)\s(said)', '"hi" John said, "yo!" told Joan! said Steve')
names2 = {name[0] for name in dialogue}

titles = re.findall('(Mr\.|Mrs\.|Ms\.|Dr\.)\s([A-Z]\w+)', '"Hello" said Dr. Strangeglove to Ms. Piggy')
names3 = {name[1] for name in titles}


print(names1.union(names2).union(names3))

{'Joan', 'John', 'Steve', 'Strangeglove', 'Piggy'}


Now, let's extend the function above to include this richer set of regular expressions


In [31]:
def find_characters(book_url):
    book_text = read_url(book_url)
    names1 = {name[1] for name in re.findall('(said|told) ([A-Z]\w+)', book_text)}
    names2 = {name[0] for name in re.findall('([A-Z]\w+)\s(said)', book_text)}
    names3 = {name[1] for name in re.findall('(Mr\.|Mrs\.|Ms\.|Dr\.)\s([A-Z]\w+)', book_text)}
    
    # names5 = {name[0] for name in re.findall('([A-Z]\w+)\s(says)', book_text)}
    
    names = names1.union(names2).union(names3)
    #names.remove('He')
    #names.remove('It')
    #names.remove('You')
    #names.remove('We')
    #names.remove('They')
    
    counts = [len(re.findall("(%s)\s" % name, book_text)) for name in names]
    characters = Table().with_columns('Name', names, 'Count', counts)
    return characters.sort('Count', descending=True)

url = "https://www.gutenberg.org/files/76/76-0.txt"
characters = find_characters(url)
characters

Name,Count
Jim,255
Tom,175
Aunt,56
Jane,30
Sawyer,30
Uncle,28
Buck,22
Everybody,19
Peter,18
Sherburn,12


## Your turn!

Your task is to now take this starter code and package up the regular expressions into a function that you can use to find the characters in the same book(s) you chose for Lab 02. 

* First apply it to your Lab 02 book(s)

* Then, apply it to some new books. 

* How well does it work? Describe the results and any false positives or negatives. If needed you can use Wikipedia or even [Sparknotes](https://www.sparknotes.com/) to determine the true cast of characters in the books.

Next, extend the function and regular expressions in order to do a better job pulling out the names in the book. 

* Write a new function that changes the regular expressions with the goal of more accurately finding a book's characters. 

* What are some of the errors that were made? How did you attempt to fix these errors? 

* Give a description of your findings together with an informal analysis of how accurate your final method is.






In [20]:
# Lab 2 book - The Count of Monte Cristo, using given code
def find_characters(book_url):
    book_text = read_url(book_url)
    names = {name for name in re.findall('said ([A-Z]\w+)', book_text)}
    counts = [len(re.findall("(%s)\s" % name, book_text)) for name in names]
    characters = Table().with_columns('Name', names, 'Count', counts)
    return characters.sort('Count', descending=True)

url = "https://www.gutenberg.org/files/1184/1184-0.txt"
characters = find_characters(url)
characters

Name,Count
Monte,1135
Madame,548
Villefort,456
Danglars,386
Dantès,378
Albert,338
Morrel,335
Franz,318
Valentine,263
Mademoiselle,194


In [21]:
#  Lab 2 book - A Christmas Carol, using given code
christmas_carol_url = "https://www.gutenberg.org/files/46/46-0.txt"
characters = find_characters(christmas_carol_url)
characters

Name,Count
Scrooge,164
Bob,24
Tiny,22
Cratchit,16
Peter,10
Fezziwig,9
Joe,2
Yes,2
Fred,0
Mrs,0


In [22]:
# Other book - Jane Eyre, using given code
def find_characters(book_url):
    book_text = read_url(book_url)
    names = {name for name in re.findall('said ([A-Z]\w+)', book_text)}
    counts = [len(re.findall("(%s)\s" % name, book_text)) for name in names]
    characters = Table().with_columns('Name', names, 'Count', counts)
    return characters.sort('Count', descending=True)

jane_eyre_url = "http://www.gutenberg.org/cache/epub/1260/pg1260.txt"
characters = find_characters(jane_eyre_url)
characters

Name,Count
Miss,284
Adele,80
Bessie,64
Diana,50
Hannah,20
Mason,19
Colonel,18
Leah,18
Eliza,17
Lord,11


The result seems not have too many false negative, but false positive such that it seperates character's first name and last name is common. For example, it will tokenize "Miss Mary Smith" into three words.

I put the revised function in Chunk 11, and will take the Count of Monte Cristo as an example to illustrate my approach of improving the revised function.

In [51]:
# Using given function (I copied this chunk from above for comparison purpose)
def find_characters(book_url):
    book_text = read_url(book_url)
    names = {name for name in re.findall('said ([A-Z]\w+)', book_text)}
    counts = [len(re.findall("(%s)\s" % name, book_text)) for name in names]
    characters = Table().with_columns('Name', names, 'Count', counts)
    return characters.sort('Count', descending=True)

url = "https://www.gutenberg.org/files/1184/1184-0.txt"
characters = find_characters(url)
characters

Name,Count
Monte,1135
Madame,548
Villefort,456
Danglars,386
Dantès,378
Albert,338
Morrel,335
Franz,318
Valentine,263
Mademoiselle,194


The first error that was made was that the names2 output contain personal nouns such as "He","She", etc (It's not obvious here, but was obvious using the example book). Based on orginal function I wrote above, I added a few lines using remove() function to remove them.  

However, there were still errors. The second error was that the first name and last name was count seperately. For example, "Madame","Danglars" was treated as two characters and counted. We can also see that the first name we extracted is "Monte", however, "Monte" also stands for an island name in this book. I revised names1 and names2 to let it count the full name and it gives the following results.

In [52]:
def find_characters(book_url):
    book_text = read_url(book_url)
    names1 = {name[1] for name in re.findall('(said|told) ([A-Z]\w+ [A-Z]\w+)', book_text)}
    names2 = {name[0] for name in re.findall('([A-Z]\w+ ?[A-Z]\w+)\s(said)', book_text)}
    names3 = {name[1] for name in re.findall('(Mr\.|Mrs\.|Ms\.|Dr\.)\s([A-Z]\w+)', book_text)}
    names = names1.union(names2).union(names3)
   # names.remove('He')
   # names.remove('She')
   # names.remove('You')
   # names.remove('They')
  
    counts = [len(re.findall("(%s)\s" % name, book_text)) for name in names]
    characters = Table().with_columns('Name', names, 'Count', counts)
    return characters.sort('Count', descending=True)

url = "https://www.gutenberg.org/files/1184/1184-0.txt"
characters = find_characters(url)
characters

Name,Count
Monte Cristo,534
Madame Danglars,89
Mademoiselle Danglars,31
La Carconte,19
Lord Wilmore,10
Madame Morrel,9
Procureur,3
King Charles,2
Gregory,2
Guillotin,1


Although it seem look better, but it doesn't consider first name only cases. So we revised the names1, names2 again and get the following result.

In [70]:
def find_characters(book_url):
    book_text = read_url(book_url)
    names1 = {name[1] for name in re.findall('(said|told) ([A-Z]\w+ ?[A-Z]?\w+ ?[A-Z]?\w+)', book_text)}
    names2 = {name[0] for name in re.findall('([A-Z]\w+ ?[A-Z]?\w+)\s(said)', book_text)}
    names3 = {name[1] for name in re.findall('(Mr\.|Mrs\.|Ms\.|Dr\.)\s([A-Z]\w+)', book_text)}
    names = names1.union(names2).union(names3)
    names.remove('It is')
    names.remove('It was')
    names.remove('She')
    names.remove('You')
    names.remove('They')
  
    counts = [len(re.findall("(%s)\s" % name, book_text)) for name in names]
    characters = Table().with_columns('Name', names, 'Count', counts)
    return characters.sort('Count', descending=True)

url = "https://www.gutenberg.org/files/1184/1184-0.txt"
characters = find_characters(url)
characters

Name,Count
Monte,1135
Monte Cristo,534
Villefort,456
Danglars,386
Dantès,378
Albert,338
Morrel,335
Franz,318
Valentine,263
Andrea,167


Now it looks much better! 

In summary, we improved the original code by considering different cases of how first name and last name of characters would appear in the text, as well as removing the personal pronoun. There are some limitation of this method, because we only consider "say","said" but not other verb to extract the subject. It needs further exploration and work.

Informal analysis of how accurate my final method is:

In [73]:
url = "https://www.gutenberg.org/files/1184/1184-0.txt"
result = len(re.findall('Monte Cristo', read_url(url)))
print(result)

1130


Monte Cristo has appeared 1130 times, but it's not only used in character names in the context but also an island name. Let's try other names - how many times they appear in the text. 

In [74]:
Villefort = len(re.findall('Villefort', read_url(url)))
print(Villefort)

1092


In [77]:
Dantes = len(re.findall('Dantès', read_url(url)))
print(Dantes)

732


In [82]:
(1130-534)/1130

0.5274336283185841

In [83]:
(1092-456)/1092

0.5824175824175825

In [84]:
(732-378)/732

0.48360655737704916

Our relative accuracy is about 50%. As I mentioned, for the most part, it caused by that we didn't consider other verbs othan "say/said". It needs further exploration and work and using more NLP techniques. 